In [ ]:
# code for server
import socket
import ssl

USERNAME = "user1"
PASSWORD = "password123"

bindsocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
bindsocket.bind(('localhost', 65432))
bindsocket.listen(5)

print("[SERVER] Waiting for connection...")

context = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH)
context.load_cert_chain(certfile="cert.pem", keyfile="key.pem")

newsocket, fromaddr = bindsocket.accept()
print(f"[SERVER] Connection from {fromaddr}")

conn = context.wrap_socket(newsocket, server_side=True)

recv_auth = conn.recv(1024).decode()
if recv_auth != f"{USERNAME}:{PASSWORD}":
    conn.send(b"Authentication Failed")
    conn.close()
    exit()
else:
    conn.send(b"Authentication Successful")
    print("[SERVER] Client authenticated successfully.")

client_msg = conn.recv(1024).decode()
print(f"[SERVER] Received: {client_msg}")

response = "Hello from Server!"
conn.send(response.encode())

conn.close()

In [ ]:
#code for client
import socket
import ssl

USERNAME = "user1"
PASSWORD = "password123"

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

context = ssl.create_default_context()
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE

conn = context.wrap_socket(sock, server_hostname='localhost')
conn.connect(('localhost', 65432))

auth = f"{USERNAME}:{PASSWORD}"
conn.send(auth.encode())

auth_response = conn.recv(1024).decode()
if "Failed" in auth_response:
    print("[CLIENT] Authentication failed. Exiting.")
    conn.close()
    exit()

print("[CLIENT] Authentication successful.")

message = "Hello from Client!"
conn.send(message.encode())

server_response = conn.recv(1024).decode()
print(f"[CLIENT] Received: {server_response}")

conn.close()